<a href="https://colab.research.google.com/github/Calvince1/Colabs/blob/main/Fraud%20in%20Tax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TAX FRAUD DETECTION FOR UNDER DECLARATIONS USING UNSUPERVISED MACHINE LEARNING MODEL**

Taxation serves as an essential method for generating revenue, enabling governments to furnish essential goods and services. However, many people dont like paying taxes and put strategies to either avoid or reduce tax liability.

Some of th Strategies include:
1. Falsifying VAT input invoices
2. Double Claim of VAT inputs
3. Underdeclaration of Sales
4. Overclaim of Purchases & ID claims
5. Use of an inconsistent stock valuation method

**Research Objectives**
1. Establish the relationship between income tax turnover and sales volume declared
2. Develop a model to predict sales underdeclarations
3.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [ ]:
df_turn